<a href="https://colab.research.google.com/github/cool60334/Team_Project_Model/blob/main/MAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import glob
import os

In [2]:
import pandas as pd
import geopy.distance
from google.colab import files
import os
from google.colab import drive

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Tibame_03期_第二組團專/訓練用資料/特徵檔案') #切換該目錄
os.listdir()

Mounted at /content/drive


['raw_data',
 '宗庭',
 '博翔',
 '俊佑',
 '聖評',
 '柏崴',
 '冠文',
 '設施座標表.xlsx',
 '設施座標表Ver2.xlsx']

In [3]:
name = input("請輸入名字後兩字：")

請輸入名字後兩字：宗庭


In [4]:
f_test = pd.read_csv(str(name)+"/台北市房屋交易_finish.csv", encoding = "utf-8")
pd.set_option('display.max_columns', 100)

In [ ]:
f_test

In [6]:
f_test = f_test.drop(["交易年月日", "建築完成年月", "交易年", "建築年", "緯度", "經度"], axis=1)

In [7]:
f_test

,土地移轉總面積平方公尺,移轉層次,總樓層數,建物移轉總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,總價元,單價元平方公尺,車位移轉總面積(平方公尺),車位總價元,主建物面積,附屬建物面積,陽台面積,屋齡,年平均CPI,CPI年增率,所得中位數,鄉鎮市區_中山區,鄉鎮市區_中正區,鄉鎮市區_信義區,鄉鎮市區_內湖區,鄉鎮市區_北投區,鄉鎮市區_南港區,鄉鎮市區_士林區,鄉鎮市區_大同區,鄉鎮市區_大安區,鄉鎮市區_文山區,鄉鎮市區_松山區,鄉鎮市區_萬華區,交易標的_建物,交易標的_房地(土地+建物),交易標的_房地(土地+建物)+車位,都市土地使用分區_住,都市土地使用分區_其他都市土地使用分區,都市土地使用分區_商,建物型態_住宅大樓(11層含以上有電梯),建物型態_公寓(5樓含以下無電梯),建物型態_套房(1房1廳1衛),建物型態_店面(店鋪),建物型態_廠辦,建物型態_華廈(10層含以下有電梯),建物型態_透天厝,主要用途_住商用,主要用途_住家用,主要用途_住工用,主要用途_其他用途,主要用途_商業用,主要用途_工業用,主要用途_辦公用,主要用途_農業用,主要建材_其他主要建材,主要建材_加強磚造,主要建材_鋼筋混凝土造,主要建材_鋼骨造,主要建材_鋼骨鋼筋混凝土造,車位類別_一樓平面,車位類別_其他車位類別,車位類別_升降平面,車位類別_升降機械,車位類別_坡道平面,車位類別_坡道機械,車位類別_塔式車位,車位類別_無車位,有無管理組織_0,有無管理組織_1,電梯_0.0,電梯_1.0,固定範圍(公園),公園數量,固定範圍(博物館),博物館數量,固定範圍(圖書館),圖書館數量,固定範圍(捷運出口),捷運出口數量,固定範圍(學校),學校數量,固定範圍(消防局),消防局數量,固定範圍(禮儀),禮儀數量,固定範圍(警察局),警察局數量,固定範圍(郵局),郵局數量,固定範圍(金融),金融數量,固定範圍(醫院),醫院數量,固定範圍(診所),診所數量,固定範圍(藥局),藥局數量
0,12.75,4,4,38.93,1,1,1,1,0.013061,0.177403,0.0,0.0,38.93,0.0,0.0,51,102.31,-0.23,0.515504,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1.8,55,2.0,0,0.6,1,0.4,0,1.8,29,2.0,5,1.0,0,1.6,4,1.2,5,1.4,14,0.2,0,1.6,97,0.4,3


In [8]:
f_test_x = f_test.drop(["總價元","單價元平方公尺","車位總價元"], axis = 1)

In [9]:
f_test_y = f_test["總價元"]

In [10]:
# 載入房屋交易csv檔

f = pd.read_csv(str(name)+"/台北市房屋交易_藥局400公尺.csv", encoding = "utf-8")
pd.set_option('display.max_columns', 100)

In [11]:
# f = pd.read_csv(str(name)+"/", encoding = "utf-8")  # 讀取csv檔案

# 去除欄位(交易年月日、交易年分、建築年分)

f = f.drop(["交易年月日", "交易年", "建築年", "緯度", "經度"], axis=1)

x = f.drop(["總價元","單價元平方公尺","車位總價元"], axis = 1) # 特徵(去除總價元、單價元平方公尺、車位總價元)
y = f["總價元"]  # 目標(取得總價元)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)  # 切分資料

p = {"max_depth": range(5, 13)}  # 設定要測試的參數及測試範圍

reg = DecisionTreeRegressor()
s = GridSearchCV(reg, p)  # 測試每個參數
s.fit(x, y)  # 訓練模型

ori_depth = s.best_params_
ori_r2 = s.best_score_
ori_r2

0.7396020577884823

In [14]:
# 原資料MAE

y_pre = s.predict(x_test)  # 帶入測試用資料進行預測，得到預測值y_pre
raw_mae = mean_absolute_error(y_test, y_pre)  # 計算MAE
raw_mae

0.01337063003909187

In [15]:
# 一筆資料MAE

f_test_y_pre = s.predict(f_test_x)  # 帶入一筆測試用資料進行預測，得到預測值f_test_y_pre
one_mae = mean_absolute_error(f_test_y, f_test_y_pre)  # 計算MAE
one_mae

0.008199478891365094